## Iniciando Modelo de Entrenamiento
Para esta parte, el articulo menciona el uso de modelo orientado a Random Forest con Regresion, actualmente ya se tiene el dataset final con mas de 34000 registros y 72 caracteristicas.
Se utilizara TensorFlow corriendo con Cuda para el entrenamiento y testeo

In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

2024-11-29 22:08:09.818339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732936089.833538   13843 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732936089.837930   13843 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 22:08:09.853346: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
print(tf.__version__)

2.18.0


In [3]:
import torch

print(torch.cuda.is_available())

True


### Importamos el dataset

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

In [5]:
dataset_df = pd.read_csv("main_dataset_without_glucose.csv")
dataset_df.head()

,datetime,glucose,gender,hba1c,food2h_calorie_mean,food2h_total_carb_sum,food2h_total_carb_mean,food2h_sugar_sum,food2h_sugar_mean,food2h_protein_sum,...,vfc_NNx,vfc_pNNx,temp_mean,temp_median,temp_max,temp_min,temp_std,temp_q1,temp_q3,ritcir_indicator_ritmo_circadiano
0,2020-02-13 17:20:00,61.0,FEMALE,5.5,0.0,0.0,0.0,0.0,0.0,0.0,...,92.0,65.7,33.604500,33.61,33.65,33.50,0.027977,33.590,33.63,0.0
1,2020-02-13 17:25:00,59.0,FEMALE,5.5,0.0,0.0,0.0,0.0,0.0,0.0,...,144.0,57.1,33.171867,33.21,33.57,32.73,0.266618,32.925,33.43,1.0
2,2020-02-13 17:30:00,58.0,FEMALE,5.5,0.0,0.0,0.0,0.0,0.0,0.0,...,112.0,60.9,33.136333,33.18,33.47,32.75,0.251755,32.870,33.39,1.0
3,2020-02-13 17:35:00,59.0,FEMALE,5.5,0.0,0.0,0.0,0.0,0.0,0.0,...,37.0,60.7,33.244767,33.25,33.39,33.11,0.052050,33.210,33.27,0.0
4,2020-02-13 17:40:00,63.0,FEMALE,5.5,0.0,0.0,0.0,0.0,0.0,0.0,...,65.0,52.4,33.315067,33.34,33.43,33.21,0.068227,33.250,33.37,0.0


In [6]:
label = 'glucose'
# classes = dataset_df[label].unique().tolist()
# print(classes)

In [7]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [8]:
train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))
y_df = test_ds_pd['glucose']

24182 examples in training, 10410 examples for testing.


In [9]:
train_ds_pd["glucose"]

0         61.0
1         59.0
2         58.0
4         63.0
5         67.0
         ...  
34584    108.0
34585    101.0
34586     97.0
34588     90.0
34590     86.0
Name: glucose, Length: 24182, dtype: float64

In [10]:
import ydf
model1 = ydf.GradientBoostedTreesLearner(label=label, task=ydf.Task.REGRESSION,max_depth = 6,num_trees=1000).train(train_ds_pd)
model2 = ydf.GradientBoostedTreesLearner(label=label, task=ydf.Task.REGRESSION,max_depth = 6,num_trees=1000)

Train model on 24182 examples


"goss_alpha" set but "sampling_method" not equal to "GOSS".
"goss_beta" set but "sampling_method" not equal to "GOSS".
"selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Data spec:
Number of records: 24182
Number of columns: 69

Number of columns by type:
	NUMERICAL: 67 (97.1014%)
	CATEGORICAL: 2 (2.89855%)

Columns:

NUMERICAL: 67 (97.1014%)
	0: "glucose" NUMERICAL mean:115.102 min:40 max:258 sd:23.2887 dtype:DTYPE_FLOAT64
	3: "hba1c" NUMERICAL mean:5.75121 min:5.3 max:6.4 sd:0.27224 dtype:DTYPE_FLOAT64
	4: "food2h_calorie_mean" NUMERICAL mean:206.425 min:0 max:1360 sd:154.68 dtype:DTYPE_FLOAT64
	5: "food2h_total_carb_sum" NUMERICAL mean:18.5773 min:0 max:463 sd:30.1133 dtype:DTYPE_FLOAT64
	6: "food2h_total_carb_mean" NUMERICAL mean:26.6507 min:0 max:154.333 sd:21.4418 dtype:DTYPE_FLOAT64
	7: "food2h_sugar_sum" NUMERICAL mean:7.60429 min:0 max:150.3 sd:13.7705 dtype:DTYPE_FLOAT64
	8: "food2h_sugar_mean" NUMERICAL mean:10.9484 min:0 max:79.5583 sd:11.14

Model trained in 0:00:28.335591


	num-trees:1000 train-loss:5.700819 train-rmse:5.700819 valid-loss:11.406914 valid-rmse:11.406914
Truncates the model to 1000 tree(s) i.e. 1000  iteration(s).
Final model num-trees:1000 valid-loss:11.406914 valid-rmse:11.406914


In [11]:
model1.describe()

In [12]:
model1.print_tree()

'gender' in ['<OOD>', 'MALE'] [score=47.864 missing=True]
    ├─(pos)─ 'food8h_cantidad_comidas_' >= 2.28646 [score=21.912 missing=True]
    │        ├─(pos)─ 'eda_std' >= 0.11627 [score=18.016 missing=True]
    │        │        ├─(pos)─ 'food24h_cantidad_comidas_' >= 8.99653 [score=45.944 missing=True]
    │        │        │        ├─(pos)─ 'food8h_cantidad_comidas_' >= 6.18229 [score=27.667 missing=False]
    │        │        │        │        ├─(pos)─ value=0.18018 sd=19.622
    │        │        │        │        └─(neg)─ value=-0.9555 sd=16.919
    │        │        │        └─(neg)─ 'bvp_median' >= 4.0625 [score=43.603 missing=False]
    │        │        │                 ├─(pos)─ value=2.5843 sd=26.748
    │        │        │                 └─(neg)─ value=0.75166 sd=23.165
    │        │        └─(neg)─ 'food2h_sugar_mean' >= 6.54854 [score=18.404 missing=True]
    │        │                 ├─(pos)─ 'food24h_media_comidas_' >= 8.50505 [score=14.744 missing=True]
    │     

In [13]:
tree = model1.get_tree(tree_idx=0)
print(tree.pretty(model1.data_spec()))

'gender' in ['<OOD>', 'MALE'] [score=47.864 missing=True]
    ├─(pos)─ 'food8h_cantidad_comidas_' >= 2.28646 [score=21.912 missing=True]
    │        ├─(pos)─ 'eda_std' >= 0.11627 [score=18.016 missing=True]
    │        │        ├─(pos)─ 'food24h_cantidad_comidas_' >= 8.99653 [score=45.944 missing=True]
    │        │        │        ├─(pos)─ 'food8h_cantidad_comidas_' >= 6.18229 [score=27.667 missing=False]
    │        │        │        │        ├─(pos)─ value=0.18018 sd=19.622
    │        │        │        │        └─(neg)─ value=-0.9555 sd=16.919
    │        │        │        └─(neg)─ 'bvp_median' >= 4.0625 [score=43.603 missing=False]
    │        │        │                 ├─(pos)─ value=2.5843 sd=26.748
    │        │        │                 └─(neg)─ value=0.75166 sd=23.165
    │        │        └─(neg)─ 'food2h_sugar_mean' >= 6.54854 [score=18.404 missing=True]
    │        │                 ├─(pos)─ 'food24h_media_comidas_' >= 8.50505 [score=14.744 missing=True]
    │     

In [14]:
tree.root.pos_child.condition

NumericalHigherThanCondition(missing=True, score=21.911544799804688, attribute=13, threshold=2.2864584922790527)

In [15]:
y_pred = model1.predict(test_ds_pd)
y_pred
len(y_pred)

Engine "GradientBoostedTreesQuickScorerExtended" built


10410

In [16]:
y_test = y_df.to_numpy()
len(y_test)

10410

In [17]:
## NO APLICA DEBIDO A QUE ES UN MODELO BASADO EN REGRESION DONDE SE CALCULAR VALORES ESTIMADOS
# # Calcular la matriz de confusión
# mat_confusion = confusion_matrix(y_test, y_pred)
# print("Matriz de confusión:")
# print(mat_confusion)

# # Calcular las métricas
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# # Imprimir las métricas
# print(f"Accuracy: {accuracy:.2f}")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1 Score: {f1:.2f}")

# # Imprimir el reporte de clasificación
# print("\nReporte de clasificación:")
# print(classification_report(y_test, y_pred))

ValueError: continuous is not supported

In [18]:
# Analyse a model (e.g. partial dependence plot, variable importance)
model1.analyze(test_ds_pd)

# Benchmark the inference speed of a model
model1.benchmark(test_ds_pd)

Engine "GradientBoostedTreesQuickScorerExtended" built
Run the model with the fast engine wrapper
List plotting attribute combinations
Found 68 combination(s)
Initiate PDP accumulator
Compute partial dependence plot for 68 set of features and 3052 model call(s) per example.
1 examples scanned.
Maximum duration reached. Interrupting analysis early.
List plotting attribute combinations
Found 68 combination(s)
Initiate CEP accumulator
Compute conditional expectation plot for 68 set of features and 3052 model call(s) per example.
1 examples scanned.
Engine "GradientBoostedTreesQuickScorerExtended" built
Running 69 features on 12 threads with 1 rounds
Model self evaluation:
Number of predictions (with weights): 1
Task: REGRESSION
Loss (SQUARED_ERROR): 11.4069

RMSE: 3.37741
Default RMSE: : 0



Single-thread inference time per example: 5.880 us (microseconds)
Details: 530910 predictions in 0.000 seconds

Multi-thread inference time per example: 0.915 us (microseconds)
Details: 3060540 predictions in 0.000 seconds using 12 threads

* Measured with the C++ serving API. See model.to_cpp().

In [19]:
evaluation1 = model1.evaluate(test_ds_pd)

# Query individual evaluation metrics
print(f"test accuracy: {evaluation1.accuracy}")

# Show the full evaluation report
print("Full evaluation report:")
evaluation1

test accuracy: None
Full evaluation report:


Evaluation()

In [20]:
# Look at specific metrics
print(evaluation1.confusion_matrix)

None


In [14]:
evaluation2 = model2.cross_validation(train_ds_pd, folds=16)
evaluation2

W0000 00:00:1732855758.170684  412451 gradient_boosted_trees.cc:1841] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1732855758.170844  412451 gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1732855758.170851  412451 gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
No input feature explicitly specified. Using all the available input features.
The label "glucose" was removed from the input feature set.
Default loss set to SQUARED_ERROR
Training gradient boosted tree on 22804 example(s) and 68 feature(s).
20544 examples used for training and 2260 examples used for validation
	num-trees:1 train-loss:22.707726 train-rmse:22.707726 valid-loss:23.377316 valid-rmse:23.377316
	num-trees:2 train-loss:22.189083 train-rmse:22.189083 valid-loss:22.847960 valid-rmse:22.847960
	num-trees:1000 train-loss:5.701499 train-rmse:5.701499 valid-loss:11.948606 va

Evaluation()

In [22]:
print(evaluation2)

RMSE: 11.6169
num examples: 10410
num examples (weighted): 10410



In [23]:
evaluation2

Evaluation()

In [33]:
model1.save("glucose_predictor")

In [34]:
loaded_model = ydf.load_model("glucose_predictor")

print(f"This is a {loaded_model.name()} model.")

This is a GRADIENT_BOOSTED_TREES model.
